# Caricamento delle collezioni e collegamento al database
Il Database è composto da 4 collezioni: food, global_dataset, global_inflation, consumer_price_index.
- food: contiene i dati relativi all'inflazione del cibo per i paesi dell'est europa
    -  Ogni paese presenta un documento per ogni mese dell'anno con i relativi dati
- global_dataset: contiene i dati relativi all'inflazione per i paesi del mondo divisi per 5 categorie
    - Ogni paese presenta un documento per ogni Series Name con i relativi dati (Un singolo documento presenta  l'inflazione per ogni anno con l'anno usato come key)
- global_inflation: contiene i dati relativi all'inflazione per i paesi del mondo divisi per anno
    - Ogni paese è rappresentato da un solo documento con i valori di inflazione riportati come in global_dataset  

In [ ]:
import pymongo
import utils 
import pandas as pd
import matplotlib.pyplot as plt
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["ProgettoBD2"]
food = db["food"]
global_dataset = db["global_dataset"]
global_inflation = db["global_inflation"]
us_cpi = db["consumer_price_index"]

## Collection: Food
- Query per estrarre i paesi presenti nella collezione \
I paesi presenti nella collezione sono principalmente paesi dell'est Europa

In [ ]:
countries = food.distinct("country")
countries

## Collection: Global Dataset
Lista dei paesi presenti nella collezione

In [ ]:
countries = global_dataset.distinct("Country")
countries

L'inflazione è suddivisa in categorie.
- Query per la stampa delle categorie presenti nella collezione

In [ ]:
global_dataset.distinct("Series Name")

# Estrazione dati su Unione Europea 
Si è deciso di analizzare i paesi dell'Unione Europea presenti nella collezione global_dataset

- Estrazione dei paesi dell'Unione Europea presenti nella collezione

In [ ]:
query = {"Country Code": {"$in": ["AUT", "BEL", "BGR", "HRV", "CYP", "CZE", "DNK", "EST", "FIN", "FRA", "DEU", "GRC", "HUN", "IRL", "ITA", "LVA", "LTU", "LUX", "MLT", "NLD", "POL", "PRT", "ROU", "SVK", "SVN", "ESP", "SWE"]}}

europeanCountries = global_dataset.find(query)

europeanCountriesList = global_dataset.distinct("Country", query)
for country in europeanCountriesList:
    print(country)

# Analizzata l'inflazione dell'intera Unione Europea negli anni
 

Query che effettua la media anno per anno del tasso di inflazione nei paesi dell'unione europea di tutte le categorie.
In sostanza vengono eliminate le categorie aggregando i 5 valori per ogni anno in una media. 

In [ ]:
# Esecuzione dell'aggregazione
#result = utils.get_avg_infl_list(global_dataset, europeanCountriesList)
# Stampa dei risultati
#for doc in result:
 #   print(doc)

In [ ]:
europeanCountriesList = global_dataset.distinct("Country", query)
result = utils.get_food_inflation_list_per_year(global_dataset, europeanCountriesList)
output = {}

  # Riempimento del dizionario con i risultati
for doc in result: 
    doc.pop('_id')
    output.update(doc)

print(output)

In [ ]:
import matplotlib.pyplot as plt

# Estrazione degli anni e dei valori di inflazione dal dizionario
anni = list(output.keys())
inflazione = list(output.values())

# Creazione del grafico
plt.figure(figsize=(10, 6))
plt.plot(anni, inflazione, marker='o', label='Inflazione annuale')
plt.xlabel('Anno')
plt.ylabel('Inflazione (%)')
plt.title('Inflazione alimentare annuale nell\'Unione Europea')
plt.xticks(rotation=90)
plt.legend()
plt.show()

## Analisi dell'inflazione alimentare dei paesi del G7

In [ ]:
query = {"Country Code": {"$in": ["FRA", "DEU", "ITA", "USA", "CAN", "JPN", "GBR"]}}

g7_countries = global_dataset.find(query)

g7_countries_list = global_dataset.distinct("Country", query)
for country in g7_countries_list:
    print(country)

In [ ]:
result = utils.get_food_inflation_list_per_year(global_dataset, g7_countries_list)
output = {}

  # Riempimento del dizionario con i risultati
for doc in result: 
    doc.pop('_id')
    output.update(doc)

print(output)

In [ ]:
import matplotlib.pyplot as plt

# Estrazione degli anni e dei valori di inflazione dal dizionario
anni = list(output.keys())
inflazione = list(output.values())

# Creazione del grafico
plt.figure(figsize=(10, 6))
plt.plot(anni, inflazione, marker='o', label='Inflazione annuale')
plt.xlabel('Anno')
plt.ylabel('Inflazione (%)')
plt.title('Inflazione alimentare annuale nei paesi del G7')
plt.xticks(rotation=90)
plt.legend()
plt.show()

## Integrazione con food: primi passi
Nel Dataset Food non sono presenti molti paesi dell'Unione Europea, per questo motivo si è deciso di integrare i dati di food con quelli di global_dataset. \
Viene eseguita una query per estrarre i dati dell'inflazione dei paesi europei sul cibo da global_dataset sfruttando il campo Series Name per estrarre solo i valori inerenti il campo "Food Consumer Price Inflation". \
 Questa query è stata effettuata utilizzando i Country Code dei paesi europei. 


In [ ]:
#todo: Implementare in app come sotto funzione dell'inflazione sul cibo totale

europeanCountriesList = ["AUT", "BEL", "BGR", "HRV", "CYP", "CZE", "DNK", "EST", "FIN", "FRA", "DEU", "GRC", "HUN", "IRL", "ITA", "LVA", "LTU", "LUX", "MLT", "NLD", "POL", "PRT", "ROU", "SVK", "SVN", "ESP", "SWE"]

result = utils.get_food_infl_countries(global_dataset, europeanCountriesList)

for doc in result:
    doc.pop('_id')
    doc.pop('Indicator Type')
    doc.pop('Note')
    print(doc)


### Query MongoDB per l'integrazione dei dati atta a raggruppare i dati di food per anno
Inizialmente i dati erano raggruppati per mese, è stata preso in considerazione il valore Inflation dove presente ed è stato sommato in modo da avere il valore annuale piuttosto che mensile. \n



# Integrazione dei dati completa del data_set food con global Dataset
- Vengono estratti i paesi distinti presenti in global_dataset e in food, viene eseguita la query per ottenere l'inflazione di ogni paese relativa al campo alimentare in global_dataset attraverso il campo Series Name. \n
- Viene in seguito definita una lista dei paesi presenti nel risultato della query e vengono ad essi aggiunti i paesi mancanti presenti nel dataset food con i relativi dati 
- Sono risultati alla fine solo due paesi mancanti: Somalia e South Sudan. I valori dell'inflazione sono stati calcolati nella cella precedente effettuando una somma dei valori mensili per ottenere quelli annuali.

## Vediamo nel dettaglio la pipeline di funzioni:
  - Viene eseguita una funzione per integrare i dati di food con global dataset.
    Vengono formattati i dati di food, precedemente aventi un documento per ogni mese in
    documenti divisi per anno e paese. Formattati 'Country', 'Year', 'infl'
    Vengono poi identificati i paesi mancanti in global dataset che saranno integrati da food.
    La differenza nella formattazione dei documenti può creare fastidio nei plot, il problema stato gestito come segue:
    - I dati vengono formattati per l'operazione di plotting rimuovendo i campo non necessari e formattando i documenti in modo che contengano l'anno e il valore di inflazione in  ogni documento.
     I documenti di global dataset erano formati da un campo 'Country' e un campo per
     ogni anno con associato il valore di inflazione.
     Nei dati di food il campo 'country' viene rinominato in 'Country'
     per uniformità con gli altri documenti.

In [ ]:
country = "Italy"
    
years, inflation_values = utils.plot_food_inflation_by_country(food, global_dataset, country)

# Crea il grafico
plt.figure(figsize=(10, 6))
plt.plot(years, inflation_values, marker='o')
plt.xlabel('Year')
plt.ylabel('Inflation (%)')
plt.title(f'Annual Inflation of {country}')
plt.grid(True)
plt.show()

## Collection: Global Inflation
Lista dei paesi presenti nella collezione

In [ ]:
countries = global_inflation.distinct("country_name")
countries

#### Estrazione Paese con maggiore inflazione in base all'anno

In [ ]:
# Query per trovare il paese con la massima inflazione in un anno specifico
anno = 1991
max_inflation = utils.get_max_infl_year(global_inflation, anno)

country_max_infl = global_inflation.find_one({"_id": max_inflation.next()['_id']})
country_name = country_max_infl['country_name']

print(f"Il paese con la maggiore inflazione nel {anno} è: {country_name} con un valore di {country_max_infl[str(anno)]}%")

In [ ]:
del country_max_infl['_id']
del country_max_infl['country_name']
del country_max_infl['indicator_name']

## Plot dell'inflazione annuale per il paese con la maggiore inflazione precedentemente calcolata

In [ ]:
import matplotlib.pyplot as plt

anni = list(country_max_infl.keys())
inflazione = list(country_max_infl.values())
plt.figure(figsize=(10, 6))
plt.plot(anni, inflazione, marker='o', label='Inflazione annuale')
plt.xlabel('Anno')
plt.ylabel('Inflazione (%)')
plt.title(f'Inflazione annuale di: {country_name}')
plt.xticks(rotation=90)
plt.show()

# Inflazione media per paese

In [ ]:

country_name = "Italy"
avg_ita_infl = utils.get_avg_infl_years(global_inflation, country_name)
print(f"L'inflazione media in {country_name} dal 1980 al 2024 è {avg_ita_infl.next()['avgInflation']:.2f}%")

# Analisi dell'inflazione negli anni dato il paese 

In [ ]:
country = "Italy"
inflation_values, years = utils.get_inflation_by_country(global_inflation, country)

plt.figure(figsize=(10, 6))
plt.plot(inflation_values, years, marker='o', label='Inflazione annuale')
plt.xlabel('Anno')
plt.ylabel('Inflazione (%)')
plt.title(f'Inflazione annuale di: {country_name}')
plt.xticks(rotation=90)
plt.legend()
plt.show()

# Operazioni CRUD
## Inserimento
Sfruttando le caratteristiche NoSQL generalizziamo l'inserimento con un documento di questa forma: country_name, inflation_values, year. \n

In [ ]:
utils.insert_into_collection(global_inflation, "Sbiriguda",69.69, 2023)
#utils.insert_into_collection(food, "Sbiriguda",69.69, 2023)
#utils.insert_into_collection(global_dataset, "Sbiriguda",69.69, 2023)

## Cancellazione

La cancellazione per correttezza e integrità dei dati è stata gestita sulla web app tramite id del documento MongoDB da eliminare \n

In [ ]:
id = utils.insert_into_collection(global_dataset, "Sbiriguda",69.69, 2023)
utils.delete_document(global_dataset, id)

# Update

In [ ]:
document = {"infl":69, "Country": "Italy"}
id = "666ae4919f6ae774f47d1617"
utils.update_document(us_cpi,id, document)